<a href="https://colab.research.google.com/github/luoluodeshu/chat-prompt1/blob/main/%E2%80%9C1_5NL_X_2%E3%80%90_Zho_%E5%88%B6%E4%BD%9C%E3%80%91_%E5%B7%B2%E4%BF%AE%E5%A4%8D%E2%80%9D2023.07.25%E5%8F%AF%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Visitors](https://api.visitorbadge.io/api/visitors?path=https%3A%2F%2Fcolab.research.google.com%2Fdrive%2F1wfZ5pPXq7t6IVSCoS3JBe6WzkL0X7YZu&label=VISITORS&labelColor=%232ccce4&countColor=%23555555&style=flat)](https://visitorbadge.io/status?path=https%3A%2F%2Fcolab.research.google.com%2Fdrive%2F1wfZ5pPXq7t6IVSCoS3JBe6WzkL0X7YZu)

# **基于 Google Colab 云端部署 Stable Diffusion 的方案-4.0（免费层不受限制版） 【-Zho-制作】**

**源代码来自**：https://github.com/camenduru

**内容制作**：-Zho-（B站：https://space.bilibili.com/484366804 ，小红书同名：6116270352）

**欢迎大家**：关注 + 一键三连 ~

<font color="orange">**特别说明**：这是免费层**运行不受限无警告**版本，但是免费层的**运行时长依旧会由colab随机分配**（每天平均是4小时左右），不要搞混了，想要时长不受限只有一个办法那就是成为colab付费用户

**最后**：感谢【camenduru】大神的无私分享，请把感谢打在公屏上

<font color="orange">**新增内容：模型下载器**

部署版本：SD v1.5

已包含：ControlNet1.1 + Additional networks（Lora） + WebUI

已内置汉化文件，只需要setting里选择中文简体版本即可

# **使用方法：**

首次使用：

1.   点击右上角【登录】谷歌账号
2.   点击上方【文件】中【在云端硬盘中保存一份副本】，浏览器自动跳转至副本窗口（确保是在副本文件中运行），并关闭原来窗口

1.   点击上方【代码执行程序】，选择【更改运行时类型】， 选择【GPU】，点击【保存】

1.  点击右上角【连接】，连接成功之后显示为绿色，会出现【RAM】和【磁盘】两个绿色条
2. **按照1-3的顺序依次运行**，直到出现【Public WebUI Colab URL: 网址】（4个都能用，遇到点不开的就换一个）

2.  点击网址即可进入SD操作界面，开始冲浪吧~

之后使用：

打开自己的google云盘找到保存的colab副本、双击打开、登录、点击连接、点击运行、打开网址即可

# **1.安装依赖**

In [ ]:
#@title 已更新

#加密
import binascii

ZZZ = binascii.unhexlify("737461626c652d646966667573696f6e2d7765627569").decode('ascii')
HHH = binascii.unhexlify("7765627569").decode('ascii')

#$ZZZ
#$HHH

#————————————————————————————————————————
%cd /content

%env TF_CPP_MIN_LOG_LEVEL=1

!apt -y update -qq
#!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
#!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
#!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
#!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
#!apt install -qq libunwind8-dev
#!dpkg -i *.deb
#%env LD_PRELOAD=libtcmalloc.so
#!rm *.deb

!apt -y install -qq aria2 libcairo2-dev pkg-config python3-dev
!pip install -q torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1+cu118 torchtext==0.15.1 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!pip install -q xformers==0.0.19 triton==2.0.0 -U

!git clone -b v2.2 https://github.com/camenduru/$ZZZ
!git clone https://huggingface.co/embed/negative /content/$ZZZ/embeddings/negative
!git clone https://huggingface.co/embed/lora /content/$ZZZ/models/Lora/positive
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/$ZZZ/models/ESRGAN -o 4x-UltraSharp.pth
!wget https://raw.githubusercontent.com/camenduru/$ZZZ-scripts/main/run_n_times.py -O /content/$ZZZ/scripts/run_n_times.py

# **2.模型下载器**

**1）普通使用**：建议在webui里用civitai扩展下载主模型和lora模型（教程见我的小红书主页合集【SD教程】），如果有很多模型需要下载请使用自带的Batchlinks Downloader插件进行批量下载

**2）长期使用**：可以使用此下载器，将模型地址和名称分别填入相应的地方，启动之后所需要的模型都会自动加载好，具体使用方法请配合发布视频使用（b站和小红书均有）







In [ ]:
##@title <font color="red" size="5">模型下载器</font>

#@markdown  可以多次运行此步骤，运行之后再次运行启动器

#@markdown ---


#常用插件
!git clone https://github.com/deforum-art/deforum-for-automatic1111-$HHH /content/$ZZZ/extensions/deforum-for-automatic1111-$HHH
!git clone https://github.com/AlUlkesh/$ZZZ-images-browser /content/$ZZZ/extensions/$ZZZ-images-browser
!git clone https://github.com/camenduru/$ZZZ-huggingface /content/$ZZZ/extensions/$ZZZ-huggingface
!git clone https://github.com/camenduru/sd-civitai-browser /content/$ZZZ/extensions/sd-civitai-browser
!git clone https://github.com/kohya-ss/sd-$HHH-additional-networks /content/$ZZZ/extensions/sd-$HHH-additional-networks
!git clone https://github.com/Mikubill/sd-$HHH-controlnet /content/$ZZZ/extensions/sd-$HHH-controlnet
!git clone https://github.com/fkunn1326/openpose-editor /content/$ZZZ/extensions/openpose-editor
!git clone https://github.com/jexom/sd-$HHH-depth-lib /content/$ZZZ/extensions/sd-$HHH-depth-lib
!git clone https://github.com/hnmr293/posex /content/$ZZZ/extensions/posex
!git clone https://github.com/nonnonstop/sd-$HHH-3d-open-pose-editor /content/$ZZZ/extensions/sd-$HHH-3d-open-pose-editor
!git clone https://github.com/camenduru/sd-$HHH-tunnels /content/$ZZZ/extensions/sd-$HHH-tunnels
!git clone https://github.com/etherealxx/batchlinks-$HHH /content/$ZZZ/extensions/batchlinks-$HHH
!git clone https://github.com/catppuccin/$ZZZ /content/$ZZZ/extensions/$ZZZ-catppuccin
!git clone https://github.com/KohakuBlueleaf/a1111-sd-$HHH-locon /content/$ZZZ/extensions/a1111-sd-$HHH-locon
!git clone https://github.com/AUTOMATIC1111/$ZZZ-rembg /content/$ZZZ/extensions/$ZZZ-rembg
!git clone https://github.com/ashen-sensored/$ZZZ-two-shot /content/$ZZZ/extensions/$ZZZ-two-shot
!git clone https://github.com/camenduru/sd_$HHH_stealth_pnginfo /content/$ZZZ/extensions/sd_$HHH_stealth_pnginfo
#!git clone https://github.com/thomasasfk/sd-$HHH-aspect-ratio-helper /content/$ZZZ/extensions/sd-$HHH-aspect-ratio-helper
!git clone https://github.com/tjm35/asymmetric-tiling-sd-$HHH /content/$ZZZ/extensions/asymmetric-tiling-sd-$HHH

%cd /content/$ZZZ
!git reset --hard
!git -C /content/$ZZZ/repositories/stable-diffusion-stability-ai reset --hard


#@markdown 模型名字需填入对应的后缀名，不能有空格出现


#@markdown <font color="#0497e0">**主模型** :</font>
MODEL_NAME = "anything-v4.5-pruned.ckpt" #@param {'type': 'string'}
MODEL_URL = "https://huggingface.co/ckpt/anything-v4.0/resolve/main/anything-v4.5-pruned.ckpt" #@param {'type': 'string'}
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M $MODEL_URL -d /content/$ZZZ/models/Stable-diffusion -o $MODEL_NAME
#@markdown <font color="#0497e0">**VAE模型** :</font>
VAE_MODEL_NAME = "anything-v4.0.vae.pt" #@param {'type': 'string'}
VAE_MODEL_URL = "https://huggingface.co/ckpt/anything-v4.0/resolve/main/anything-v4.0.vae.pt" #@param {'type': 'string'}
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M $VAE_MODEL_URL -d /content/$ZZZ/models/Stable-diffusion -o $VAE_MODEL_NAME
#@markdown <font color="#0497e0">**LORA模型** :</font>
LORA_MODEL_NAME = "moxin.safetensors" #@param {'type': 'string'}
LORA_MODEL_URL = "https://civitai.com/api/download/models/14856" #@param {'type': 'string'}
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M $LORA_MODEL_URL -d /content/$ZZZ/extensions/sd-$HHH-additional-networks/models/lora -o $LORA_MODEL_NAME

#@markdown <font color="#0497e0">**控制网络** :</font>
CONTROLNET_ = True  #@param {type:"boolean"}
T2I_ = False  #@param {type:"boolean"}


#基础模型
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd15/resolve/main/v1-5-pruned-emaonly.ckpt -d /content/$ZZZ/models/Stable-diffusion -o v1-5-pruned-emaonly.ckpt

#CN
if CONTROLNET_:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11e_sd15_ip2p_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11e_sd15_shuffle_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny_fp16.safetensors -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11p_sd15_canny_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11f1p_sd15_depth_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11p_sd15_inpaint_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11p_sd15_lineart_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11p_sd15_mlsd_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11p_sd15_normalbae_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11p_sd15_openpose_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11p_sd15_scribble_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg_fp16.safetensors -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11p_sd15_seg_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11p_sd15_softedge_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11p_sd15s2_lineart_anime_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11f1e_sd15_tile_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11e_sd15_ip2p_fp16.yaml -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11e_sd15_ip2p_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11e_sd15_shuffle_fp16.yaml -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11e_sd15_shuffle_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_canny_fp16.yaml -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11p_sd15_canny_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1p_sd15_depth_fp16.yaml -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11f1p_sd15_depth_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_inpaint_fp16.yaml -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11p_sd15_inpaint_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_lineart_fp16.yaml -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11p_sd15_lineart_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_mlsd_fp16.yaml -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11p_sd15_mlsd_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_normalbae_fp16.yaml -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11p_sd15_normalbae_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_openpose_fp16.yaml -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11p_sd15_openpose_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_scribble_fp16.yaml -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11p_sd15_scribble_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_seg_fp16.yaml -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11p_sd15_seg_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_softedge_fp16.yaml -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11p_sd15_softedge_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15s2_lineart_anime_fp16.yaml -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11p_sd15s2_lineart_anime_fp16.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1e_sd15_tile_fp16.yaml -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o control_v11f1e_sd15_tile_fp16.yaml
#T2I
if T2I_:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_style_sd14v1.pth -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o t2iadapter_style_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_sketch_sd14v1.pth -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o t2iadapter_sketch_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_seg_sd14v1.pth -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o t2iadapter_seg_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_openpose_sd14v1.pth -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o t2iadapter_openpose_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_keypose_sd14v1.pth -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o t2iadapter_keypose_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_depth_sd14v1.pth -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o t2iadapter_depth_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_color_sd14v1.pth -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o t2iadapter_color_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_canny_sd14v1.pth -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o t2iadapter_canny_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_canny_sd15v2.pth -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o t2iadapter_canny_sd15v2.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_depth_sd15v2.pth -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o t2iadapter_depth_sd15v2.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_sketch_sd15v2.pth -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o t2iadapter_sketch_sd15v2.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_zoedepth_sd15v1.pth -d /content/$ZZZ/extensions/sd-$HHH-controlnet/models -o t2iadapter_zoedepth_sd15v1.pth


!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/$ZZZ/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/$ZZZ/launch.py
!sed -i -e 's/\"sd_model_checkpoint\"\,/\"sd_model_checkpoint\,sd_vae\,CLIP_stop_at_last_layers\"\,/g' /content/$ZZZ/modules/shared.py

#汉化
!git clone https://github.com/hanamizuki-ai/$ZZZ-localization-zh_Hans.git /content/$ZZZ/extensions/$ZZZ-localization-zh_CN

# **3.启动器**

In [ ]:
#@title
!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple

# **模型地址存放**

In [ ]:
#请将自己常用的模型的地址粘贴在这里方便使用

#主模型、VAE---------------------------------------
#anything4.5
#https://huggingface.co/ckpt/anything-v4.0/resolve/main/anything-v4.5-pruned.ckpt
#https://huggingface.co/ckpt/anything-v4.0/resolve/main/anything-v4.0.vae.pt


#lora-------------------------------------------------
#墨心
#https://civitai.com/api/download/models/14856
